# RAG using SaaS LLM - This notebook contains sample code for using AzureAI, Bedrock or Gemeni as LLM module. Please customize based on your specific use case

Example Usage: | makeresults
| eval query = "How to create an ivestigation on Splunk ES?"
| fit MLTKContainer algo=llm_rag_script_saas_llm embedder_name="all-MiniLM-L6-v2" embedder_dimension=384 collection_name="document_collection_splunk" top_k=4 rag_type=Documents query into app:llm_rag_script_saas_llm as RAG


## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [3]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import pymilvus
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
import llama_index
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document, StorageContext, ServiceContext
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.llms.ollama import Ollama
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.llms.openai import OpenAI
from llama_index.llms.bedrock import Bedrock
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import textwrap
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate


# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"
LLM_ENDPOINT = "http://ollama:11434"

In [4]:
# Some logging settings 
import logging
import sys
import llama_index.core
from llama_index.core.callbacks import (
    CallbackManager,
    LlamaDebugHandler,
    CBEventType,
)

llama_index.core.set_global_handler("simple")

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.26.4
pandas version: 2.2.0


## Stage 1

In [6]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

## Stage 2 - create and initialize a model

In [10]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 42.0
    return model

## Stage 3 - fit the model

In [19]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    info = {"message": "model trained"}
    return info

In [13]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model trained'}


## Stage 4 - apply the model

In [1]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    # "Documents" or "Logs"
    try:
        d_type = param['options']['params']['rag_type'].strip('\"')
    except:
        d_type = "Documents"
    
    X = df["query"].values.tolist()
    use_local= int(param['options']['params']['use_local'])
    try:
        embedder_name = param['options']['params']['embedder_name'].strip('\"')
    except:
        embedder_name = 'all-MiniLM-L6-v2'

    try:
        collection_name = param['options']['params']['collection_name'].strip('\"')
    except:
        collection_name = "default-doc-collection"

    if embedder_name == 'intfloat/multilingual-e5-large':
        embedder_dimension = 1024
    elif embedder_name == 'all-MiniLM-L6-v2':
        embedder_dimension = 384
    else:
        try:
            embedder_dimension = int(param['options']['params']['embedder_dimension'])
        except:
            embedder_dimension = 384
    if use_local:
        embedder_name = f'/srv/app/model/data/{embedder_name}'
        print("Using local embedding model checkpoints")
    try:
        top_k = int(param['options']['params']['top_k'])
    except:
        top_k = 5
        
    if d_type == "Documents":
        qa_prompt_str = (
            "Below are the context information.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information as well as necessary prior knowledge, "
            "answer the question: {query_str}\n"
        )
        chat_text_qa_msgs = [
            (
                "system",
                "You are an expert Q&A system that is trusted around the world. Always answer the query using the provided context information and reasoning as detailed as possible",
            ),
            ("user", qa_prompt_str),
        ]
    else:
        qa_prompt_str = (
            "Past log messages below are given as context information.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information as well as necessary prior knowledge, "
            "answer the question: {query_str}\n"
        )
        chat_text_qa_msgs = [
            (
                "system",
                "You are an expert Q&A system that is trusted around the world. Always answer the query using the provided context information and reasoning as detailed as possible",
            ),
            ("user", qa_prompt_str),
        ]
    
    text_qa_template = ChatPromptTemplate.from_messages(chat_text_qa_msgs)

    # Example for SaaS LLM service integration
    llm_service = ""
    if llm_service == "azure":
        ## Azure AI Example
        api_key = "XXXXXXXXXXXXXXXX"
        azure_endpoint = "https://XXXX.openai.azure.com"
        api_version = "API-VERSION"
        
        llm = AzureOpenAI(
            model="MODEL_NAME",
            deployment_name="DEPLOYMENT_NAME",
            api_key=api_key,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
        )
    elif llm_service == "bedrock":
        ## Bedrock Example
        llm = Bedrock(
            model="amazon.titan-text-express-v1",
            aws_access_key_id="AWS Access Key ID to use",
            aws_secret_access_key="AWS Secret Access Key to use",
            aws_session_token="AWS Session Token to use",
            aws_region_name="AWS Region to use, eg. us-east-1",
        )
    elif llm_service == "gemini":
        ## Gemeni Example
        GOOGLE_API_KEY = "YOUR GOOGLE API KEY"  
        os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
        llm = Gemini()
    else:
        llm = None

    try:
        transformer_embedder = HuggingFaceEmbedding(model_name=embedder_name)
        service_context = ServiceContext.from_defaults(
            llm=llm, embed_model=transformer_embedder, chunk_size=1024
        )
    except:
        cols = {"Response": ["ERROR: Could not load embedder"], "References": ["ERROR"]}
        result = pd.DataFrame(data=cols)
        return result
    try:
        if d_type == "Documents":
            vector_store = MilvusVectorStore(uri="http://milvus-standalone:19530", token="", collection_name=collection_name, dim=embedder_dimension, overwrite=False)
        else:
            vector_store = MilvusVectorStore(uri="http://milvus-standalone:19530", token="", collection_name=collection_name, embedding_field='embeddings', text_key='label', dim=embedder_dimension, overwrite=False)
        index = VectorStoreIndex.from_vector_store(
           vector_store=vector_store, service_context=service_context
        )
        query_engine = index.as_query_engine(similarity_top_k=top_k, text_qa_template=text_qa_template)
    except:
        cols = {"Response": ["ERROR: Could not load collection"], "References": ["ERROR"]}
        result = pd.DataFrame(data=cols)
        return result
        
    l = []
    f = []
    for i in range(len(X)):
        r = query_engine.query(X[i])
        l.append(r.response)
        if d_type == "Documents":
            files = ""
            for node in r.source_nodes:
                files += node.node.metadata['file_path']
                files += "\n"
            f.append(files)
        else:
            logs = ""
            for i in range(len(r.source_nodes)):
                logs += r.source_nodes[0].text
                logs += "\n"
            f.append(logs)       
    
    cols = {"Response": l, "References": f}
    result = pd.DataFrame(data=cols)
    return result

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(None,None,None))

## Stage 5 - save the model

In [16]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [17]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [18]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

def compute(model,df,param):
    # "Documents" or "Logs"
    try:
        d_type = param['params']['rag_type'].strip('\"')
    except:
        d_type = "Documents"
    
    X = df[0]["query"]
    use_local= int(param['params']['use_local'])
    try:
        embedder_name = param['params']['embedder_name'].strip('\"')
    except:
        embedder_name = 'all-MiniLM-L6-v2'

    try:
        collection_name = param['params']['collection_name'].strip('\"')
    except:
        collection_name = "default-doc-collection"

    if embedder_name == 'intfloat/multilingual-e5-large':
        embedder_dimension = 1024
    elif embedder_name == 'all-MiniLM-L6-v2':
        embedder_dimension = 384
    else:
        try:
            embedder_dimension = int(param['params']['embedder_dimension'])
        except:
            embedder_dimension = 384
    if use_local:
        embedder_name = f'/srv/app/model/data/{embedder_name}'
        print("Using local embedding model checkpoints")
    try:
        top_k = int(param['params']['top_k'])
    except:
        top_k = 5
        
    if d_type == "Documents":
        qa_prompt_str = (
            "Below are the context information.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information as well as necessary prior knowledge, "
            "answer the question: {query_str}\n"
        )
        chat_text_qa_msgs = [
            (
                "system",
                "You are an expert Q&A system that is trusted around the world. Always answer the query using the provided context information and reasoning as detailed as possible",
            ),
            ("user", qa_prompt_str),
        ]
    else:
        qa_prompt_str = (
            "Past log messages below are given as context information.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information as well as necessary prior knowledge, "
            "answer the question: {query_str}\n"
        )
        chat_text_qa_msgs = [
            (
                "system",
                "You are an expert Q&A system that is trusted around the world. Always answer the query using the provided context information and reasoning as detailed as possible",
            ),
            ("user", qa_prompt_str),
        ]
    
    text_qa_template = ChatPromptTemplate.from_messages(chat_text_qa_msgs)

    # Example for SaaS LLM service integration
    llm_service = ""
    if llm_service == "azure":
        ## Azure AI Example
        api_key = "XXXXXXXXXXXXXXXX"
        azure_endpoint = "https://XXXX.openai.azure.com"
        api_version = "API-VERSION"
        
        llm = AzureOpenAI(
            model="MODEL_NAME",
            deployment_name="DEPLOYMENT_NAME",
            api_key=api_key,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
        )
    elif llm_service == "bedrock":
        ## Bedrock Example
        llm = Bedrock(
            model="amazon.titan-text-express-v1",
            aws_access_key_id="AWS Access Key ID to use",
            aws_secret_access_key="AWS Secret Access Key to use",
            aws_session_token="AWS Session Token to use",
            aws_region_name="AWS Region to use, eg. us-east-1",
        )
    elif llm_service == "gemini":
        ## Gemeni Example
        GOOGLE_API_KEY = "YOUR GOOGLE API KEY"  
        os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
        llm = Gemini()
    else:
        llm = None

    try:
        transformer_embedder = HuggingFaceEmbedding(model_name=embedder_name)
        service_context = ServiceContext.from_defaults(
            llm=llm, embed_model=transformer_embedder, chunk_size=1024
        )
    except:
        cols = {"Response": "ERROR: Could not load embedder", "References": "ERROR"}
        result = [cols]
        return result
    try:
        if d_type == "Documents":
            vector_store = MilvusVectorStore(uri="http://milvus-standalone:19530", token="", collection_name=collection_name, dim=embedder_dimension, overwrite=False)
        else:
            vector_store = MilvusVectorStore(uri="http://milvus-standalone:19530", token="", collection_name=collection_name, embedding_field='embeddings', text_key='label', dim=embedder_dimension, overwrite=False)
        index = VectorStoreIndex.from_vector_store(
           vector_store=vector_store, service_context=service_context
        )
        query_engine = index.as_query_engine(similarity_top_k=top_k, text_qa_template=text_qa_template)
    except:
        cols = {"Response": "ERROR: Could not load collection", "References": "ERROR"}
        result = [cols]
        return result
        

    
    r = query_engine.query(X)
    l = r.response
    if d_type == "Documents":
        files = ""
        for node in r.source_nodes:
            files += node.node.metadata['file_path']
            files += "\n"
    else:
        files = ""
        for i in range(len(r.source_nodes)):
            files += r.source_nodes[0].text
            files += "\n"     
    
    cols = {"Response": l, "References": files}
    result = [cols]
    return result

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [18]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [19]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [20]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


In [21]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [22]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


## End of Stages
All subsequent cells are not tagged and can be used for further freeform code